In [1]:
# Check to see if we're using a GPU
!nvidia-smi

/bin/bash: nvidia-smi: command not found


# Imports

In [2]:
import tensorflow as tf
import pandas as pd
import re # RegEx for removing non-letter characters
import nltk # Natural language processing
nltk.download("stopwords")
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from tqdm.notebook import tqdm_notebook
from nltk.stem.porter import PorterStemmer

!pip install textblob
from textblob import TextBlob

!pip install wordcloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Connect to google drive

Connecting to google drive in order to have access to MyDrive to save the best models when fitting and also load the necessary data.

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH="/content/drive/MyDrive/machine_learning_bitcoin_sentiment_analysis"

Mounted at /content/drive


# Get the data

In [4]:
# Download bitcoin data from github
!wget https://raw.githubusercontent.com/NegreaAlexandruCristian/Lucrare-Dizertatie-ML/main/btc_more_data.csv

# Download bitcoin tweets from github
!wget https://raw.githubusercontent.com/NegreaAlexandruCristian/Lucrare-Dizertatie-ML/main/bitcoin_tweets.csv

--2023-04-19 07:31:44--  https://raw.githubusercontent.com/NegreaAlexandruCristian/Lucrare-Dizertatie-ML/main/btc_more_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 285282 (279K) [text/plain]
Saving to: ‘btc_more_data.csv’

btc_more_data.csv   100%[===================>] 278.60K  --.-KB/s    in 0.01s   

2023-04-19 07:31:45 (22.3 MB/s) - ‘btc_more_data.csv’ saved [285282/285282]

--2023-04-19 07:31:45--  https://raw.githubusercontent.com/NegreaAlexandruCristian/Lucrare-Dizertatie-ML/main/bitcoin_tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, aw

In [5]:
# Load Twitter dataset
df_tweets = pd.read_csv('/content/bitcoin_tweets.csv', error_bad_lines=False,
                        lineterminator='\n')
df_tweets.head()

<ipython-input-5-f79d294c127d>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_tweets = pd.read_csv('/content/bitcoin_tweets.csv', error_bad_lines=False,


,Unnamed: 0,DateTime,Text,Language,ReplyCount,RetweetCount,LikeCount,QuoteCount
0,0,2013-04-28 23:59:17+00:00,Actu: http://t.co/8cvjA69kRA will soon accept ...,en,0,0,0,0
1,1,2013-04-28 23:59:17+00:00,"Actu: A list of bitcoin hostile banks, please ...",en,0,0,0,0
2,2,2013-04-28 23:59:14+00:00,0.10 Bitcoin(s) (1/10) Btc (10 % Of A Bitcoin)...,en,0,0,0,0
3,3,2013-04-28 23:59:13+00:00,http://t.co/1Hf8C3FZou will soon accept Bitcoi...,en,0,0,0,0
4,4,2013-04-28 23:59:13+00:00,"A list of bitcoin hostile banks, please contri...",en,0,0,0,0


# Preprocess data (starting with twitter data)

In [6]:
df_tweets.count

<bound method DataFrame.count of         Unnamed: 0                   DateTime  \
0                0  2013-04-28 23:59:17+00:00   
1                1  2013-04-28 23:59:17+00:00   
2                2  2013-04-28 23:59:14+00:00   
3                3  2013-04-28 23:59:13+00:00   
4                4  2013-04-28 23:59:13+00:00   
...            ...                        ...   
364195      364195  2023-04-17 23:50:14+00:00   
364196      364196  2023-04-17 23:50:09+00:00   
364197      364197  2023-04-17 23:50:09+00:00   
364198      364198  2023-04-17 23:50:04+00:00   
364199      364199  2023-04-17 23:50:00+00:00   

                                                     Text Language  \
0       Actu: http://t.co/8cvjA69kRA will soon accept ...       en   
1       Actu: A list of bitcoin hostile banks, please ...       en   
2       0.10 Bitcoin(s) (1/10) Btc (10 % Of A Bitcoin)...       en   
3       http://t.co/1Hf8C3FZou will soon accept Bitcoi...       en   
4       A list of bitcoin ho

In [7]:
df_tweets.head()

,Unnamed: 0,DateTime,Text,Language,ReplyCount,RetweetCount,LikeCount,QuoteCount
0,0,2013-04-28 23:59:17+00:00,Actu: http://t.co/8cvjA69kRA will soon accept ...,en,0,0,0,0
1,1,2013-04-28 23:59:17+00:00,"Actu: A list of bitcoin hostile banks, please ...",en,0,0,0,0
2,2,2013-04-28 23:59:14+00:00,0.10 Bitcoin(s) (1/10) Btc (10 % Of A Bitcoin)...,en,0,0,0,0
3,3,2013-04-28 23:59:13+00:00,http://t.co/1Hf8C3FZou will soon accept Bitcoi...,en,0,0,0,0
4,4,2013-04-28 23:59:13+00:00,"A list of bitcoin hostile banks, please contri...",en,0,0,0,0


In [8]:
df_tweets.tail()

,Unnamed: 0,DateTime,Text,Language,ReplyCount,RetweetCount,LikeCount,QuoteCount
364195,364195,2023-04-17 23:50:14+00:00,"And here is Grayscale #Bitcoin, same pattern, ...",en,2,1,14,0
364196,364196,2023-04-17 23:50:09+00:00,Big win for all,en,15,8,35,2
364197,364197,2023-04-17 23:50:09+00:00,My country needs Bitcoin,en,0,0,0,0
364198,364198,2023-04-17 23:50:04+00:00,This zesty faucet from @_bitcoiner is making m...,en,0,0,1,0
364199,364199,2023-04-17 23:50:00+00:00,Beaxy Exchange Explains Why it Trusts Bitcoin ...,en,0,0,0,0


In [9]:
df_tweets['DateTime'] = df_tweets['DateTime'].astype(str)
max(df_tweets['DateTime']), min(df_tweets['DateTime'])

('2023-04-17 23:59:55+00:00', '2013-04-28 23:13:08+00:00')

## Get the dataframe and drop the unecessary columns

In [10]:
df_tweets = df_tweets.drop(['Language' , 'ReplyCount' ,'RetweetCount',
                           'LikeCount', 'QuoteCount', 'Unnamed: 0'], axis=1)
df_tweets.head()

,DateTime,Text
0,2013-04-28 23:59:17+00:00,Actu: http://t.co/8cvjA69kRA will soon accept ...
1,2013-04-28 23:59:17+00:00,"Actu: A list of bitcoin hostile banks, please ..."
2,2013-04-28 23:59:14+00:00,0.10 Bitcoin(s) (1/10) Btc (10 % Of A Bitcoin)...
3,2013-04-28 23:59:13+00:00,http://t.co/1Hf8C3FZou will soon accept Bitcoi...
4,2013-04-28 23:59:13+00:00,"A list of bitcoin hostile banks, please contri..."


In [11]:
# Convert the 'date' column to datetime format
df_tweets['DateTime'] = pd.to_datetime(df_tweets['DateTime'])

# Format the datetime object as a string with the 'yyyy-mm-dd' format
df_tweets['DateTime'] = df_tweets['DateTime'].dt.strftime('%Y-%m-%d')

In [12]:
# Rename the columns
df_tweets = df_tweets.rename(columns={'DateTime': 'Date', 'Text': 'tweet_content'})
df_tweets

,Date,tweet_content
0,2013-04-28,Actu: http://t.co/8cvjA69kRA will soon accept ...
1,2013-04-28,"Actu: A list of bitcoin hostile banks, please ..."
2,2013-04-28,0.10 Bitcoin(s) (1/10) Btc (10 % Of A Bitcoin)...
3,2013-04-28,http://t.co/1Hf8C3FZou will soon accept Bitcoi...
4,2013-04-28,"A list of bitcoin hostile banks, please contri..."
...,...,...
364195,2023-04-17,"And here is Grayscale #Bitcoin, same pattern, ..."
364196,2023-04-17,Big win for all
364197,2023-04-17,My country needs Bitcoin
364198,2023-04-17,This zesty faucet from @_bitcoiner is making m...


In [13]:
df_tweets=df_tweets[['tweet_content']]

## Calculate for each tweet the polarity and subjectivity

We need this in order to feed the model data so that it can learn. Tensorflow models loves numbers between 0 and 1. But we will get to the numbers later when we have the data preprared and ready.

### Clean Text from tweet

In [14]:
def clean_text(tweet):
    '''
    Convert tweet text into a sequence of words that don't have stop words 
    '''

    # Convert to lowercase
    text = tweet.lower()
    # Remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Removing @ mention
    text = re.sub(r'@[A-Za-z0-9]+','',text) 
    # Removing # symbol
    text = re.sub(r'#','',text) 
    # Removing RT followed byspace
    text = re.sub(r'RT[\s]+','',text) 
    # Removing https
    text = re.sub(r'https?:\/\/\S+','',text)

    words = text.split()

    # Remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]

    # Apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    sentence = ' '.join(words)

    return sentence

In [ ]:
# Extract the words that are the most relevant for the model to learn.
# Eliminate stop words and non english words
# Note: Not all the words will be relevant for the model
cleantext=[]
for item in tqdm_notebook(df_tweets['tweet_content']):
    words=clean_text(item)
    cleantext+=[words]
df_tweets['cleantext']=cleantext
df_tweets

  0%|          | 0/364200 [00:00<?, ?it/s]

In [ ]:
df_tweets.head(10)

In [ ]:
# Create function to subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
# Create function to get polarity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Create two new columns
df_tweets['subjectivity']=df_tweets['cleantext'].apply(getSubjectivity)
df_tweets['polarity']=df_tweets['cleantext'].apply(getPolarity)

# Show new Dataframe
df_tweets

In [ ]:
# Drop the tweet content since it's no longer needed
# We have now the Subjectivity and Polarity of the text
df_tweets = df_tweets.drop(['tweet_content'], axis=1)

In [ ]:
df_tweets.head(10)

In [ ]:
df_tweets.info()

In [ ]:
# Group the tweets by dates and see how much volume we have for each day
df = df_tweets.groupby(pd.Grouper(key='Date',freq='D')).size().reset_index(name='tweet_vol')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# The index of the dataframe of tweets should be the date
df.index = pd.to_datetime(df.index)
df = df.set_index('Date')

In [ ]:
df.head()

In [ ]:
# Calculate the mean for everything since all of the data is grouped
df_tweets.index = pd.to_datetime(df_tweets.index)
df['tweet_vol'] = df['tweet_vol'].astype(float)
df_clean = df_tweets.groupby('DateTime').agg(lambda x: x.mean())
df_clean['Tweet_vol'] = df['tweet_vol']

In [ ]:
df_clean = df.drop(df.index[0])
df_clean.head()

In [ ]:
# Get the data of bitcoin inside the DataFrame
btc_df = pd.read_csv('/content/btc_more_data.csv', error_bad_lines=False, engine = 'python')

In [ ]:
# Make the index of the bitcoin DataFrame the date
btc_df['Date'] = pd.to_datetime(btc_df['Date'])
btc_df = btc_df.set_index(pd.DatetimeIndex(btc_df['Date']))